In [1]:
import pickle
import pandas as pd
from graphdatascience import GraphDataScience
import torch
from torch_geometric.transforms import RandomNodeSplit
import random
import numpy as np
from neo4j import GraphDatabase
from torch_geometric.data import Data
import os

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set seeds for consistent results
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [3]:
def load(file):
    with open(file, 'rb') as f:
        return pickle.load(f)
    
def save(data, file):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

In [4]:
neo4j = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'admin'))
gds = GraphDataScience.from_neo4j_driver(neo4j)
device = torch.device('cuda')

def run_query(query, params={}):
    with neo4j.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [5]:
def normalize_topology_index(new_idx_to_old, topology):
    old_idx_to_new = dict((v, k) for k, v in new_idx_to_old.items())
    return [[old_idx_to_new[node_id] for node_id in nodes] for nodes in topology]

def load_graph(theory_name):
    print("loading " + theory_name)
    run_query(f"""CALL gds.graph.drop("{theory_name}", false)""")
    gds.graph.project.cypher(
        theory_name,
        f"""
        MATCH (n:Position)-[:in]->(t:Theory)
        WHERE t.id = "{theory_name}"
        RETURN id(n) AS id, id(n) as label, apoc.node.degree(n) as degree
        """, f"""
        MATCH (ln:Entity)-[:in]->(n:Position)-[:in]->(n1:Theory)
        WHERE n1.id = "{theory_name}"
        WITH n AS src, ln AS src_e
        MATCH (src_e)-[r]-(ln:Entity)-[:in]->(n:Position)-[:in]->(n1:Theory)
        WHERE n1.id = "{theory_name}"
        RETURN id(src) AS source, id(n) AS target
        """)
    graph = gds.graph.get(theory_name)
    
    topology = gds.beta.graph.relationships.stream(graph).by_rel_type()
    node_props = gds.graph.nodeProperties.stream(graph, ['degree'], separate_property_columns=True)
    
    if topology == {}:
        topology = {'__ALL__': []}
    normalized_topology = normalize_topology_index(dict(node_props["nodeId"]), topology["__ALL__"])
    
    edge_index = torch.tensor(normalized_topology, dtype=torch.long)
    x = torch.tensor(node_props["degree"], dtype=torch.float).view([-1, 1])
    data_line = data[data['theory'] == theory_name].iloc[0]
    y = torch.tensor([data_line['class']], dtype=torch.long)
    
    res = Data(x=x, y=y, edge_index=edge_index)
    res['theory'] = theory_name
    res['frequency'] = data_line['frequency']
    gds.graph.drop(graph)
    if len(topology["__ALL__"]) == 0:
        return None
    else:
        return res.to(device)

# Load data from db/graph_features

In [6]:
data = pd.read_csv('data/graph_features.csv.gz')
data['class'] = (data['frequency'] > 0.01).astype(int)
manual = pd.read_csv('data/manual_quality.csv')
manual_data = manual.merge(data, on='theory')
quality = pd.read_csv('data/theory_quality_pred.csv')
quality_data = quality.merge(manual, on='theory')
for theory in manual['theory'].values:
    if theory not in manual_data['theory'].values:
        print(f'Missing {theory}')
display(data)

,theory,sloc,out_min,out_max,out_avg,out_med,eigen_min,eigen_max,eigen_avg,eigen_med,...,between_min,between_max,between_avg,between_med,cluster_min,cluster_max,cluster_avg,cluster_med,frequency,class
0,ADS_Functor.Merkle_Interface,198,0,22,3.187500,3.0,0.0,8.503687e-43,1.328701e-44,1.233283e-233,...,0.0,483.555491,21.420794,3.000000,0.000000,1.000000,0.339453,0.333333,0.005051,0
1,ADS_Functor.ADS_Construction,895,0,6620,93.525424,4.0,0.0,1.756936e-03,1.112859e-05,2.791846e-310,...,0.0,461.940323,39.806280,0.010190,0.000000,2.000000,0.333575,0.318432,0.000000,0
2,ADS_Functor.Generic_ADS_Construction,307,1,11316,465.800000,12.5,0.0,6.574049e-01,3.096136e-02,2.571684e-64,...,0.0,124.193812,13.501176,2.100072,0.000000,1.500000,0.482062,0.428571,0.000000,0
3,ADS_Functor.Inclusion_Proof_Construction,330,0,286,33.781609,6.0,0.0,1.024810e-35,1.806505e-37,8.590296e-312,...,0.0,152.185935,8.467260,0.000000,0.000000,1.000000,0.493100,0.466667,0.000000,0
4,ADS_Functor.Canton_Transaction_Tree,361,0,1711,75.464646,12.5,0.0,6.053158e-12,9.413334e-14,4.562393e-57,...,0.0,1036.867696,33.141623,0.013160,0.000000,1.000000,0.428947,0.409545,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7326,pGCL.Determinism,158,4,41,10.161290,6.0,0.0,1.343013e-303,1.678766e-304,0.000000e+00,...,0.0,3735.151414,292.452248,31.067715,0.090909,0.500000,0.330459,0.333333,0.006329,0
7327,pGCL.Loops,124,0,29,11.000000,6.0,0.0,1.678766e-304,2.582717e-305,0.000000e+00,...,0.0,470.824956,55.524296,0.142857,0.116809,0.600000,0.368135,0.400000,0.008065,0
7328,pGCL.Termination,269,12,45,27.000000,22.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,154.060308,66.992567,28.592769,0.056025,0.220779,0.133580,0.110661,0.011152,1
7329,pGCL.Primitives,58,0,804,116.000000,2.0,0.0,4.648205e-01,4.451763e-02,0.000000e+00,...,0.0,13.976048,1.529412,0.000000,0.000000,1.100000,0.519888,0.500000,0.000000,0


In [7]:
neo4j_theories = run_query("MATCH (t:Theory) RETURN t.id AS theory")
display(neo4j_theories)

,theory
0,CCL.Set
1,CCL.Lfp
2,CCL.Gfp
3,CCL.CCL
4,CCL.Trancl
...,...
8620,ZF-ex.Ramsey
8621,ZF-ex.Limit
8622,ZF-ex.BinEx
8623,ZF-ex.LList


In [8]:
graphs = [load_graph(theory) for theory in neo4j_theories['theory'].values if theory in data['theory'].values]
graphs = [graph for graph in graphs if graph is not None]
random.shuffle(graphs)

loading CCL.Set
loading CCL.Lfp
loading CCL.Gfp
loading CCL.CCL
loading CCL.Trancl
loading CCL.Term
loading CCL.Type
loading CCL.Hered
loading CCL.Wfd
loading CCL.Fix
loading CCL.Nat
loading CCL.List
loading CCL.Stream
loading CCL.Flag
loading CTT.CTT
loading CTT.Equality
loading Cube.Cube
loading IFOL
loading FOL
loading IFOLP
loading FOLP
loading HOL.HOL


Loading: 100% 100.0/100 [00:00<00:00, 389.07%/s]


loading HOL.Orderings
loading HOL.Groups
loading HOL.Lattices
loading HOL.Boolean_Algebras
loading HOL.Set
loading HOL.Typedef
loading HOL.Fun
loading HOL.Complete_Lattices
loading HOL.Ctr_Sugar
loading HOL.Inductive
loading HOL.Product_Type
loading HOL.Sum_Type
loading HOL.Rings
loading HOL.Nat
loading HOL.Fields
loading HOL.Finite_Set
loading HOL.Relation
loading HOL.Transitive_Closure
loading HOL.Wellfounded
loading HOL.Wfrec
loading HOL.Order_Relation
loading HOL.Hilbert_Choice
loading HOL.Zorn
loading HOL.BNF_Wellorder_Relation
loading HOL.BNF_Wellorder_Embedding
loading HOL.BNF_Wellorder_Constructions
loading HOL.BNF_Cardinal_Order_Relation
loading HOL.BNF_Cardinal_Arithmetic
loading HOL.BNF_Def
loading HOL.BNF_Composition
loading HOL.Basic_BNFs
loading HOL.BNF_Fixpoint_Base
loading HOL.BNF_Least_Fixpoint
loading HOL.Equiv_Relations
loading HOL.Basic_BNF_LFPs
loading HOL.Meson
loading HOL.ATP
loading HOL.Metis
loading HOL.Transfer
loading HOL.Lifting
loading HOL.Quotient
loading 

Loading: 100% 100.0/100 [00:00<00:00, 256.75%/s]


loading Promela.PromelaDatastructures


...

loading CakeML.TypeSystem
loading CakeML.TypeSystemAuxiliary
loading CakeML.Semantic_Extras
loading CakeML.Evaluate_Termination
loading CakeML.Evaluate_Clock
loading CakeML.Evaluate_Single
loading CakeML.Big_Step_Determ
loading CakeML.Big_Step_Total
loading CakeML.Big_Step_Fun_Equiv
loading CakeML.Big_Step_Unclocked
loading CakeML.Big_Step_Clocked
loading CakeML.Big_Step_Unclocked_Single
loading CakeML.Matching
loading CakeML.CakeML_Code
loading CakeML.CakeML_Quickcheck
loading CakeML_Codegen.ML_Utils
loading CakeML_Codegen.Compiler_Utils
loading CakeML_Codegen.CakeML_Utils
loading CakeML_Codegen.Test_Utils
loading CakeML_Codegen.Terms_Extras
loading CakeML_Codegen.HOL_Datatype
loading CakeML_Codegen.Constructors
loading CakeML_Codegen.Consts
loading CakeML_Codegen.Strong_Term
loading CakeML_Codegen.Sterm
loading CakeML_Codegen.Pterm
loading CakeML_Codegen.Term_as_Value
loading CakeML_Codegen.Value
loading CakeML_Codegen.CupCake_Env
loading CakeML_Codegen.CupCake_Semantics
loading Cake

Loading: 100% 100.0/100 [00:00<00:00, 309.84%/s]


loading SPARCv8.Lib
loading SPARCv8.DetMonad
loading SPARCv8.DetMonadLemmas
loading SPARCv8.RegistersOps
loading SPARCv8.MMU
loading SPARCv8.Sparc_State
loading SPARCv8.Sparc_Instruction
loading SPARCv8.Sparc_Execution
loading SPARCv8.Sparc_Properties
loading SPARCv8.Sparc_Init_State
loading Security_Protocol_Refinement.Infra
loading Security_Protocol_Refinement.Refinement
loading Security_Protocol_Refinement.Agents
loading Security_Protocol_Refinement.Keys
loading Security_Protocol_Refinement.Atoms
loading Security_Protocol_Refinement.Runs
loading Security_Protocol_Refinement.Channels
loading Security_Protocol_Refinement.Message
loading Security_Protocol_Refinement.s0g_secrecy
loading Security_Protocol_Refinement.a0n_agree
loading Security_Protocol_Refinement.a0i_agree
loading Security_Protocol_Refinement.m1_auth
loading Security_Protocol_Refinement.m2_auth_chan
loading Security_Protocol_Refinement.m2_confid_chan
loading Security_Protocol_Refinement.m3_sig
loading Security_Protocol_Re

Loading: 100% 100.0/100 [00:00<00:00, 598.24%/s]


loading IP_Addresses.Prefix_Match
loading IP_Addresses.CIDR_Split
loading IP_Addresses.WordInterval_Sorted
loading IP_Addresses.IP_Address_Parser
loading IP_Addresses.Lib_Numbers_toString
loading IP_Addresses.Lib_Word_toString
loading IP_Addresses.Lib_List_toString
loading IP_Addresses.IP_Address_toString
loading IP_Addresses.Prefix_Match_toString
loading Simple_Firewall.Lib_Enum_toString
loading Simple_Firewall.L4_Protocol
loading Simple_Firewall.Simple_Packet
loading Simple_Firewall.Firewall_Common_Decision_State
loading Simple_Firewall.Iface
loading Simple_Firewall.SimpleFw_Syntax
loading Simple_Firewall.SimpleFw_Semantics
loading Simple_Firewall.List_Product_More
loading Simple_Firewall.Option_Helpers
loading Simple_Firewall.Generic_SimpleFw
loading Simple_Firewall.Shadowed
loading Simple_Firewall.IP_Partition_Preliminaries
loading Simple_Firewall.GroupF
loading Simple_Firewall.IP_Addr_WordInterval_toString
loading Simple_Firewall.Primitives_toString
loading Simple_Firewall.Service

# Compute Splits

In [9]:
print(f"{len(graphs)} Graphs, e.g. {graphs[0]}")
train_dataset = graphs[:int(len(graphs)*0.70)]
validate_dataset = graphs[int(len(graphs)*0.70):int(len(graphs)*0.85)]
test_dataset = graphs[int(len(graphs)*0.85):]
if os.path.exists('data/train'):
    assert [graph['theory'] for graph in train_dataset] == load('data/train')
    assert [graph['theory'] for graph in validate_dataset] == load('data/validate')
    assert [graph['theory'] for graph in test_dataset] == load('data/test')
else:
    save([graph['theory'] for graph in train_dataset], 'data/train')
    save([graph['theory'] for graph in validate_dataset], 'data/validate')
    save([graph['theory'] for graph in test_dataset], 'data/test')
len(train_dataset), len(validate_dataset), len(test_dataset)

7089 Graphs, e.g. Data(x=[27, 1], edge_index=[2, 468], y=[1], theory='CAVA_LTL_Modelchecker.CAVA_Abstract', frequency=0.0062111801242236)


(4962, 1063, 1064)

# Pickle/Store Data

In [10]:
save(train_dataset, 'data/train_graphs')
train = data[data['theory'].isin([graph['theory'] for graph in train_dataset])]
save(train, 'data/train_classical')
train[['theory', 'frequency']].to_csv('data/train.csv', index=False)

save(validate_dataset, 'data/validate_graphs')
validate = data[data['theory'].isin([graph['theory'] for graph in validate_dataset])]
save(validate, 'data/validate_classical')
validate[['theory', 'frequency']].to_csv('data/validate.csv', index=False)

save(test_dataset, 'data/test_graphs')
test = data[data['theory'].isin([graph['theory'] for graph in test_dataset])]
save(test, 'data/test_classical')
test[['theory', 'frequency']].to_csv('data/test.csv', index=False)

In [11]:
manual_graphs = list()
for graph in graphs:
    if graph['theory'] in manual_data['theory'].values:
        graph = Data.clone(graph)
        data_line = manual_data[manual_data['theory'] == graph['theory']].iloc[0]
        graph.y = torch.tensor([data_line['bad']], dtype=torch.long)
        manual_graphs.append(graph)
save(manual_graphs, 'data/manual_graphs')
save(manual_data, 'data/manual_data')
save(quality_data, 'data/quality_data')